In [ ]:
!pip install -r requirements.txt

In [ ]:
from collections.abc import Callable
from enum import Enum

import numpy as np

class Method(Enum):
    ALTERNATING_DIRECTION = 0,
    FRACTIONAL_STEP = 1

def splitting_method(alpha: tuple, f_x_y_t: Callable,
    l: tuple, n: tuple t: float, k_upper: int,
    alpha1: float, beta1: float, alpha2: float, beta2: float,
    alpha3: float, beta3: float, alpha4: float, beta4: float,
    phi_1_x_t, phi_2_x_t, phi_3_y_t, phi_4_y_t, psi_x_y,
    method: Method) -> np.ndarray:
    h, tau, sigma = l / n, t / k_upper, alpha * tau / (h * h)

    ai = np.full(n[0] + 1, sigma_a / (2.0 if method == ALTERNATING_DIRECTION else 1.0))
    bi = np.full(n[0] + 1, -(1.0 if method == ALTERNATING_DIRECTION else 2.0) * sigma_a - 1.0)
    ci = np.full(n[0] + 1, sigma_a / (2.0 if method == ALTERNATING_DIRECTION else 1.0))
    aj = np.full(n[1] + 1, sigma_b / (2.0 if method == ALTERNATING_DIRECTION else 1.0))
    bj = np.full(n[1] + 1, -(1.0 if method == ALTERNATING_DIRECTION else 2.0) * sigma_b - 1.0)
    cj = np.full(n[1] + 1, sigma_b / (2.0 if method == ALTERNATING_DIRECTION else 1.0))
    di, dj = np.zeros(n[0] + 1), np.zeros(n[1] + 1)

    ai[0] = c_i[n[0]] = a_j[0] = c_j[n[1]] = 0.0
    bi[0] = beta3 - alpha3 / h1
    ci[0] = alpha3 / h1
    ai[n[0]] = -alpha4 / h1
    bi[n[0]] = beta4 + alpha4 / h1
    bj[0] = beta1 - alpha1 / h2
    cj[0] = alpha1 / h2
    aj[n[1]] = -alpha2 / h2
    bj[n[1]] = beta2 + alpha2 / h2

    u, u_temp = np.zero((2, n_1 + 1, n_2 + 1))
    __initial_approximation(n[0], h1, n[1], h2, psi_x_y, u)
    for k in range(1, k_upper + 1):
        step_x(f_x_y_t,
            n[0], h1, n[1], h2, tau, k, sigma_b, alpha1, beta1, alpha2, beta2,
            phi_1_x_t, phi_2_x_t, phi_3_y_t, phi_4_y_t, ai, bi, ci, di,
            method, u, u_temp)
        step_y(f_x_y_t,
            n[0], h1, n[1], h2, tau, k, sigma_a, alpha3, beta3, alpha4, beta4,
            phi_1_x_t, phi_2_x_t, phi_3_y_t, phi_4_y_t, aj, bj, cj, dj,
            method, u_temp, u)
    return u


def __initial_approximation(n[0], h1, n[1], h2, psi_x_y, u0):
    for i in range(n[0] + 1):
        for j in range(n[1] + 1):
            u0[i, j] = psi_x_y(i * h_1, j * h_2)

def step_x(f_x_y_t,
    n[0]: int, h1: float, n[1]: int, h2: float,
    tau: float, k: int, sigma_b: float,
    alpha1: float, beta1: float, alpha2: float, beta2: float,
    phi_1_x_t, phi_2_x_t, phi_3_y_t, phi_4_y_t,
    ai: np.ndarray, bi: np.ndarray, ci: np.ndarray, di: np.ndarray,
    method: Method, u_k_minus_1: np.ndarray, u_k_minus_1_divides_2: np.ndarray):
    for j in range(1, n[1]):
        di[0] = phi_3_y_t(j * h2, (k - 0.5) * tau)
        d_i[n_1] = phi_4_y_t(j * h_2, (k - 0.5) * tau)
        for i in range(1, n[0]):
            di[i] = (-u_k_minus_1[i, j]
                - tau / 2.0 * f_x_y_t(a, b, mu, i * h_1, j * h_2, (k - 0.5) * tau))
            if method == ALTERNATING_DIRECTION:
                di[i] += (sigma_b * u_k_minus_1[i, j]
                    - sigma_b / 2.0 * u_k_minus_1[i, j - 1]
                    - sigma_b / 2.0 * u_k_minus_1[i, j + 1])
        ublas::column(u_k_minus_1_divides_2, j) = thomas_algorithm(a_i, b_i, c_i, d_i);
    for (std::size_t i = 0; i <= n_1; ++i)
    {
        u_k_minus_1_divides_2(i, 0) = (-alpha_1 * u_k_minus_1_divides_2(i, 1)
                + h_2 * phi_1_x_t(a, b, mu, i * h_1, (k - 0.5) * tau))
            / (beta_1 * h_2 - alpha_1);
        u_k_minus_1_divides_2(i, n_2) = (alpha_2 * u_k_minus_1_divides_2(i, n_2 - 1)
                + h_2 * phi_2_x_t(a, b, mu, i * h_1, (k - 0.5) * tau))
            / (beta_2 * h_2 + alpha_2);
    }
}

template<typename T, typename>
static void step_y(const T a, const T b, const T mu,
    const std::function<T (const T &, const T &, const T &,
        const T &, const T &, const T &)> &f_x_y_t,
    const std::size_t n_1, const T h_1, const std::size_t n_2, const T h_2,
    const T tau, const std::size_t k, const T sigma_a,
    const T alpha_3, const T beta_3, const T alpha_4, const T beta_4,
    const std::function<T (const T &, const T &, const T &, const T &, const T &)> &phi_1_x_t,
    const std::function<T (const T &, const T &, const T &, const T &, const T &)> &phi_2_x_t,
    const std::function<T (const T &, const T &, const T &, const T &, const T &)> &phi_3_y_t,
    const std::function<T (const T &, const T &, const T &, const T &, const T &)> &phi_4_y_t,
    const ublas::vector<T> &a_j, const ublas::vector<T> &b_j, const ublas::vector<T> &c_j,
    ublas::vector<T> &d_j, const Method method, const ublas::matrix<T> &u_k_minus_1_divides_2,
    ublas::matrix<T> &u_k)
{
    for (std::size_t i = 1; i < n_1; ++i)
    {
        d_j(0) = phi_1_x_t(a, b, mu, i * h_1, k * tau);
        d_j(n_2) = phi_2_x_t(a, b, mu, i * h_1, k * tau);
        for (std::size_t j = 1; j < n_2; ++j)
        {
            d_j(j) = -u_k_minus_1_divides_2(i, j)
                - tau / 2.0 * f_x_y_t(a, b, mu, i * h_1, j * h_2, k * tau);
            if (method == ALTERNATING_DIRECTION)
            {
                d_j(j) += sigma_a * u_k_minus_1_divides_2(i, j)
                    - sigma_a / 2.0 * u_k_minus_1_divides_2(i - 1, j)
                    - sigma_a / 2.0 * u_k_minus_1_divides_2(i + 1, j);
            }
        }
        ublas::row(u_k, i) = thomas_algorithm(a_j, b_j, c_j, d_j);
    }
    for (std::size_t j = 0; j <= n_2; ++j)
    {
        u_k(0, j) = (-alpha_3 * u_k(1, j)
            + h_1 * phi_3_y_t(a, b, mu, j * h_2, k * tau)) / (beta_3 * h_1 - alpha_3);
        u_k(n_1, j) = (alpha_4 * u_k(n_1 - 1, j)
            + h_1 * phi_4_y_t(a, b, mu, j * h_2, k * tau)) / (beta_4 * h_1 + alpha_4);
    }
}

In [ ]:
u, u_temp = np.zeros((2, 5, 7))
a, b = np.arange(4, 6)